In [25]:
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation
import os


## Combine the ego and obj files based on timestamps

In [5]:
# Read the CSV files into dataframes
# utm_corr = pd.read_csv('utm_corr.csv')
# yolo_obj = pd.read_csv('yolo_obj.csv')

ego_xy = pd.read_csv('utm_corr.csv')
obj_xy = pd.read_csv('yolo_obj.csv')

# # Convert timestamp and rosbagTimestamp columns to string
# utm_corr['rosbagTimestamp'] = utm_corr['rosbagTimestamp'].astype(str)
# yolo_obj['timestamp'] = yolo_obj['timestamp'].astype(str)

# # Remove the common prefix from timestamp and rosbagTimestamp
# utm_corr['rosbagTimestamp'] = utm_corr['rosbagTimestamp'].str.replace('168148', '')
# yolo_obj['timestamp'] = yolo_obj['timestamp'].str.replace('168148', '')


# Create a new dataframe to store the combined results
combined_df = pd.DataFrame(columns=['obj_timestamp', 'ego_timestamp', 'secs', 'nsecs', 'obj_x', 'obj_y', 'obj_z', 'label', 'ego_x', 'ego_y', 'ego_z', 'ego_ox', 'ego_oy', 'ego_oz', 'ego_ow'])

In [7]:
# Iterate over each timestamp in obj_xy and find the nearest timestamp in ego_xy
for index, row in obj_xy.iterrows():
    obj_timestamp = row['timestamp']
    ego_timestamp = ego_xy['rosbagTimestamp'].values[np.abs(ego_xy['rosbagTimestamp'].values - obj_timestamp).argmin()]
    ego_row = ego_xy[ego_xy['rosbagTimestamp'] == ego_timestamp]
    combined_row = {
        'obj_timestamp': obj_timestamp,
        'ego_timestamp': ego_row['rosbagTimestamp'].values[0],
        'secs': ego_row['secs'].values[0],
        'nsecs': ego_row['nsecs'].values[0],
        'obj_x': row['position_x'],
        'obj_y': row['position_y'],
        'obj_z': row['position_z'],
        'label': row['label'],
        'ego_x': ego_row['position_x'].values[0],
        'ego_y': ego_row['position_y'].values[0],
        'ego_z': ego_row['position_z'].values[0],
        'ego_ox': ego_row['orientation_x'].values[0],
        'ego_oy': ego_row['orientation_y'].values[0],
        'ego_oz': ego_row['orientation_z'].values[0],
        'ego_ow': ego_row['orientation_w'].values[0]
        }
    combined_df = combined_df.append(combined_row, ignore_index=True)


combined_df

C:\Users\ylinz\AppData\Local\Temp\ipykernel_19168\1743228455.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(combined_row, ignore_index=True)
C:\Users\ylinz\AppData\Local\Temp\ipykernel_19168\1743228455.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(combined_row, ignore_index=True)
C:\Users\ylinz\AppData\Local\Temp\ipykernel_19168\1743228455.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(combined_row, ignore_index=True)
C:\Users\ylinz\AppData\Local\Temp\ipykernel_19168\1743228455.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

,obj_timestamp,ego_timestamp,secs,nsecs,obj_x,obj_y,obj_z,label,ego_x,ego_y,ego_z,ego_ox,ego_oy,ego_oz,ego_ow
0,1681488521572679936.0,1681488521576580096.0,1681488521.0,537272027.0,13.509211,4.060675,-2.194613,7.0,722748.1434,3382476.076,130.740482,0.015598,0.026357,-0.996713,0.074998
1,1681488603933939968.0,1681488603940559872.0,1681488603.0,901910094.0,67.571983,8.5723,-1.584964,7.0,722443.6243,3383240.372,120.213521,-0.011228,0.002161,-0.823207,-0.567626
2,1681488604034149888.0,1681488604030810112.0,1681488604.0,30675227.0,63.920612,8.279163,-1.499821,7.0,722443.1083,3383241.72,120.263848,-0.011494,0.001544,-0.823327,-0.56745
3,1681488604134350080.0,1681488604132630016.0,1681488604.0,117585984.0,60.14579,7.939707,-1.4117,7.0,722442.4885,3383243.337,120.321945,-0.012461,0.001137,-0.823363,-0.567377
4,1681488604234530048.0,1681488604244559872.0,1681488604.0,203770432.0,56.400768,7.555552,-1.324138,7.0,722442.0737,3383244.415,120.360779,-0.012506,0.001103,-0.823429,-0.56728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,1681489018846119936.0,1681489018844559872.0,1681489018.0,836400577.0,59.419807,4.238417,1.73368,11.0,721826.0,3389550.0,98.0,0.0,0.0,1.0,1.0
300,1681489019347120128.0,1681489019336600064.0,1681489019.0,336471474.0,56.689102,11.02952,2.352881,11.0,721827.0,3389551.0,98.0,0.0,0.0,1.0,1.0
301,1681489019647759872.0,1681489019636559872.0,1681489019.0,601597892.0,55.130283,16.225712,1.337258,11.0,721827.0,3389552.0,98.0,0.0,0.0,1.0,1.0
302,1681488845907579904.0,1681488845901850112.0,1681488845.0,901733093.0,50.526253,15.408875,-3.385401,7.0,722237.0,3386906.0,111.0,0.0,0.0,-1.0,-1.0


### Truncation error when comparing rosbagTimestamp! delete the first six digit before comparing them. (168148.....)

In [8]:
# Read the CSV files into dataframes
# utm_corr = pd.read_csv('utm_corr.csv')
# yolo_obj = pd.read_csv('yolo_obj.csv')

ego_xy = pd.read_csv('utm_corr_trunc.csv')
obj_xy = pd.read_csv('yolo_obj_trunc.csv')

# # Convert timestamp and rosbagTimestamp columns to string
# utm_corr['rosbagTimestamp'] = utm_corr['rosbagTimestamp'].astype(str)
# yolo_obj['timestamp'] = yolo_obj['timestamp'].astype(str)

# # Remove the common prefix from timestamp and rosbagTimestamp
# utm_corr['rosbagTimestamp'] = utm_corr['rosbagTimestamp'].str.replace('168148', '')
# yolo_obj['timestamp'] = yolo_obj['timestamp'].str.replace('168148', '')


# Create a new dataframe to store the combined results
combined_df = pd.DataFrame(columns=['obj_timestamp', 'ego_timestamp', 'secs', 'nsecs', 'obj_x', 'obj_y', 'obj_z', 'label', 'ego_x', 'ego_y', 'ego_z', 'ego_ox', 'ego_oy', 'ego_oz', 'ego_ow'])

In [10]:
# Iterate over each timestamp in obj_xy and find the nearest timestamp in ego_xy
for index, row in obj_xy.iterrows():
    obj_timestamp = row['timestamp']
    ego_timestamp = ego_xy['rosbagTimestamp'].values[np.abs(ego_xy['rosbagTimestamp'].values - obj_timestamp).argmin()]
    ego_row = ego_xy[ego_xy['rosbagTimestamp'] == ego_timestamp]
    combined_row = {
        'obj_timestamp': obj_timestamp,
        'ego_timestamp': ego_row['rosbagTimestamp'].values[0],
        'secs': ego_row['secs'].values[0],
        'nsecs': ego_row['nsecs'].values[0],
        'obj_x': row['position_x'],
        'obj_y': row['position_y'],
        'obj_z': row['position_z'],
        'label': row['label'],
        'ego_x': ego_row['position_x'].values[0],
        'ego_y': ego_row['position_y'].values[0],
        'ego_z': ego_row['position_z'].values[0],
        'ego_ox': ego_row['orientation_x'].values[0],
        'ego_oy': ego_row['orientation_y'].values[0],
        'ego_oz': ego_row['orientation_z'].values[0],
        'ego_ow': ego_row['orientation_w'].values[0]
        }
    combined_df = combined_df.append(combined_row, ignore_index=True)


combined_df

C:\Users\ylinz\AppData\Local\Temp\ipykernel_19168\1743228455.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(combined_row, ignore_index=True)
C:\Users\ylinz\AppData\Local\Temp\ipykernel_19168\1743228455.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(combined_row, ignore_index=True)
C:\Users\ylinz\AppData\Local\Temp\ipykernel_19168\1743228455.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(combined_row, ignore_index=True)
C:\Users\ylinz\AppData\Local\Temp\ipykernel_19168\1743228455.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

,obj_timestamp,ego_timestamp,secs,nsecs,obj_x,obj_y,obj_z,label,ego_x,ego_y,ego_z,ego_ox,ego_oy,ego_oz,ego_ow
0,8521572680000.0,8521576580000.0,1681488521.0,537272027.0,13.509211,4.060675,-2.194613,7.0,722748.1434,3382476.076,130.740482,0.015598,0.026357,-0.996713,0.074998
1,8603933940000.0,8603940560000.0,1681488603.0,901910094.0,67.571983,8.5723,-1.584964,7.0,722443.6243,3383240.372,120.213521,-0.011228,0.002161,-0.823207,-0.567626
2,8604034150000.0,8604030810000.0,1681488604.0,30675227.0,63.920612,8.279163,-1.499821,7.0,722443.1083,3383241.72,120.263848,-0.011494,0.001544,-0.823327,-0.56745
3,8604134350000.0,8604132630000.0,1681488604.0,117585984.0,60.14579,7.939707,-1.4117,7.0,722442.4885,3383243.337,120.321945,-0.012461,0.001137,-0.823363,-0.567377
4,8604234530000.0,8604244560000.0,1681488604.0,203770432.0,56.400768,7.555552,-1.324138,7.0,722442.0737,3383244.415,120.360779,-0.012506,0.001103,-0.823429,-0.56728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,9018846120000.0,9018844560000.0,1681489018.0,836400577.0,59.419807,4.238417,1.73368,11.0,721826.0,3389550.0,98.0,0.0,0.0,1.0,1.0
300,9019347120000.0,9019336600000.0,1681489019.0,336471474.0,56.689102,11.02952,2.352881,11.0,721827.0,3389551.0,98.0,0.0,0.0,1.0,1.0
301,9019647760000.0,9019636560000.0,1681489019.0,601597892.0,55.130283,16.225712,1.337258,11.0,721827.0,3389552.0,98.0,0.0,0.0,1.0,1.0
302,8845907580000.0,8845901850000.0,1681488845.0,901733093.0,50.526253,15.408875,-3.385401,7.0,722237.0,3386906.0,111.0,0.0,0.0,-1.0,-1.0


In [11]:
combined_df.to_csv('ego_obj_combined.csv', index=False)


## Compute the rotation matrix for each timestamp

In [13]:
ego_obj = pd.read_csv('ego_obj_combined.csv')
ego_obj

,obj_timestamp,ego_timestamp,secs,nsecs,obj_x,obj_y,obj_z,label,ego_x,ego_y,ego_z,ego_ox,ego_oy,ego_oz,ego_ow
0,8.521573e+12,8.521577e+12,1.681489e+09,537272027.0,13.509211,4.060675,-2.194613,7.0,722748.1434,3382476.076,130.740482,0.015598,0.026357,-0.996713,0.074998
1,8.603934e+12,8.603941e+12,1.681489e+09,901910094.0,67.571983,8.572300,-1.584964,7.0,722443.6243,3383240.372,120.213521,-0.011228,0.002161,-0.823207,-0.567626
2,8.604034e+12,8.604031e+12,1.681489e+09,30675227.0,63.920612,8.279163,-1.499821,7.0,722443.1083,3383241.720,120.263848,-0.011494,0.001544,-0.823327,-0.567450
3,8.604134e+12,8.604133e+12,1.681489e+09,117585984.0,60.145790,7.939707,-1.411700,7.0,722442.4885,3383243.337,120.321945,-0.012461,0.001137,-0.823363,-0.567377
4,8.604235e+12,8.604245e+12,1.681489e+09,203770432.0,56.400768,7.555552,-1.324138,7.0,722442.0737,3383244.415,120.360779,-0.012506,0.001103,-0.823429,-0.567280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,9.018846e+12,9.018845e+12,1.681489e+09,836400577.0,59.419807,4.238417,1.733680,11.0,721826.0000,3389550.000,98.000000,0.000000,0.000000,1.000000,1.000000
300,9.019347e+12,9.019337e+12,1.681489e+09,336471474.0,56.689102,11.029520,2.352881,11.0,721827.0000,3389551.000,98.000000,0.000000,0.000000,1.000000,1.000000
301,9.019648e+12,9.019637e+12,1.681489e+09,601597892.0,55.130283,16.225712,1.337258,11.0,721827.0000,3389552.000,98.000000,0.000000,0.000000,1.000000,1.000000
302,8.845908e+12,8.845902e+12,1.681489e+09,901733093.0,50.526253,15.408875,-3.385401,7.0,722237.0000,3386906.000,111.000000,0.000000,0.000000,-1.000000,-1.000000


In [14]:
# Extract the required columns
orientations = ego_obj[['ego_ow', 'ego_ox', 'ego_oy', 'ego_oz']]
obj_x = ego_obj['obj_x']
obj_y = ego_obj['obj_y']
obj_z = ego_obj['obj_z']



# Iterate through the rows and apply rotation to obj_x, obj_y, obj_z
obj_x_t = []
obj_y_t = []
obj_z_t = []
for i in range(len(ego_obj)):
    quaternion = orientations.iloc[i].values
    rotation = Rotation.from_quat(quaternion)
    rotation_matrix = rotation.as_matrix()
    obj_vector = np.array([obj_x.iloc[i], obj_y.iloc[i], obj_z.iloc[i]])
    transformed_vector = np.dot(rotation_matrix, obj_vector)
    obj_x_t.append(transformed_vector[0])
    obj_y_t.append(transformed_vector[1])
    obj_z_t.append(transformed_vector[2])

# Add the transformed coordinates as new attributes in the DataFrame
ego_obj['obj_x_t'] = obj_x_t
ego_obj['obj_y_t'] = obj_y_t
ego_obj['obj_z_t'] = obj_z_t

ego_obj

,obj_timestamp,ego_timestamp,secs,nsecs,obj_x,obj_y,obj_z,label,ego_x,ego_y,ego_z,ego_ox,ego_oy,ego_oz,ego_ow,obj_x_t,obj_y_t,obj_z_t
0,8.521573e+12,8.521577e+12,1.681489e+09,537272027.0,13.509211,4.060675,-2.194613,7.0,722748.1434,3382476.076,130.740482,0.015598,0.026357,-0.996713,0.074998,13.766277,3.001277,-2.299158
1,8.603934e+12,8.603941e+12,1.681489e+09,901910094.0,67.571983,8.572300,-1.584964,7.0,722443.6243,3383240.372,120.213521,-0.011228,0.002161,-0.823207,-0.567626,67.668676,5.150399,6.032665
2,8.604034e+12,8.604031e+12,1.681489e+09,30675227.0,63.920612,8.279163,-1.499821,7.0,722443.1083,3383241.720,120.263848,-0.011494,0.001544,-0.823327,-0.567450,64.006705,5.020154,5.880340
3,8.604134e+12,8.604133e+12,1.681489e+09,117585984.0,60.145790,7.939707,-1.411700,7.0,722442.4885,3383243.337,120.321945,-0.012461,0.001137,-0.823363,-0.567377,60.226942,4.884737,5.603807
4,8.604235e+12,8.604245e+12,1.681489e+09,203770432.0,56.400768,7.555552,-1.324138,7.0,722442.0737,3383244.415,120.360779,-0.012506,0.001103,-0.823429,-0.567280,56.478306,4.627558,5.354717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,9.018846e+12,9.018845e+12,1.681489e+09,836400577.0,59.419807,4.238417,1.733680,11.0,721826.0000,3389550.000,98.000000,0.000000,0.000000,1.000000,1.000000,59.419807,-1.733680,4.238417
300,9.019347e+12,9.019337e+12,1.681489e+09,336471474.0,56.689102,11.029520,2.352881,11.0,721827.0000,3389551.000,98.000000,0.000000,0.000000,1.000000,1.000000,56.689102,-2.352881,11.029520
301,9.019648e+12,9.019637e+12,1.681489e+09,601597892.0,55.130283,16.225712,1.337258,11.0,721827.0000,3389552.000,98.000000,0.000000,0.000000,1.000000,1.000000,55.130283,-1.337258,16.225712
302,8.845908e+12,8.845902e+12,1.681489e+09,901733093.0,50.526253,15.408875,-3.385401,7.0,722237.0000,3386906.000,111.000000,0.000000,0.000000,-1.000000,-1.000000,50.526253,3.385401,15.408875


In [15]:
ego_obj.to_csv('obj_ego_with_transformed.csv', index=False)


## Add the transformed xyz onto the utm, (switch x and y)

In [16]:
ego_obj = pd.read_csv('obj_ego_with_transformed.csv')
ego_obj

,obj_timestamp,ego_timestamp,secs,nsecs,obj_x,obj_y,obj_z,label,ego_x,ego_y,ego_z,ego_ox,ego_oy,ego_oz,ego_ow,obj_x_t,obj_y_t,obj_z_t
0,8.521573e+12,8.521577e+12,1.681489e+09,537272027.0,13.509211,4.060675,-2.194613,7.0,722748.1434,3382476.076,130.740482,0.015598,0.026357,-0.996713,0.074998,13.766277,3.001277,-2.299158
1,8.603934e+12,8.603941e+12,1.681489e+09,901910094.0,67.571983,8.572300,-1.584964,7.0,722443.6243,3383240.372,120.213521,-0.011228,0.002161,-0.823207,-0.567626,67.668676,5.150399,6.032665
2,8.604034e+12,8.604031e+12,1.681489e+09,30675227.0,63.920612,8.279163,-1.499821,7.0,722443.1083,3383241.720,120.263848,-0.011494,0.001544,-0.823327,-0.567450,64.006705,5.020154,5.880340
3,8.604134e+12,8.604133e+12,1.681489e+09,117585984.0,60.145790,7.939707,-1.411700,7.0,722442.4885,3383243.337,120.321945,-0.012461,0.001137,-0.823363,-0.567377,60.226942,4.884737,5.603807
4,8.604235e+12,8.604245e+12,1.681489e+09,203770432.0,56.400768,7.555552,-1.324138,7.0,722442.0737,3383244.415,120.360779,-0.012506,0.001103,-0.823429,-0.567280,56.478306,4.627558,5.354717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,9.018846e+12,9.018845e+12,1.681489e+09,836400577.0,59.419807,4.238417,1.733680,11.0,721826.0000,3389550.000,98.000000,0.000000,0.000000,1.000000,1.000000,59.419807,-1.733680,4.238417
300,9.019347e+12,9.019337e+12,1.681489e+09,336471474.0,56.689102,11.029520,2.352881,11.0,721827.0000,3389551.000,98.000000,0.000000,0.000000,1.000000,1.000000,56.689102,-2.352881,11.029520
301,9.019648e+12,9.019637e+12,1.681489e+09,601597892.0,55.130283,16.225712,1.337258,11.0,721827.0000,3389552.000,98.000000,0.000000,0.000000,1.000000,1.000000,55.130283,-1.337258,16.225712
302,8.845908e+12,8.845902e+12,1.681489e+09,901733093.0,50.526253,15.408875,-3.385401,7.0,722237.0000,3386906.000,111.000000,0.000000,0.000000,-1.000000,-1.000000,50.526253,3.385401,15.408875


### Add a unified time, note: backward time

In [20]:
# Calculate the time difference from the first row
time_diff = ego_obj['secs'].iloc[0]

# Calculate the unified time attribute in seconds with the shifted time
unified_time_data = (ego_obj['secs'] - time_diff) + ego_obj['nsecs'] * 1e-9

ego_obj['time'] = unified_time_data

ego_obj

,obj_timestamp,ego_timestamp,secs,nsecs,obj_x,obj_y,obj_z,label,ego_x,ego_y,ego_z,ego_ox,ego_oy,ego_oz,ego_ow,obj_x_t,obj_y_t,obj_z_t,time
0,8.521573e+12,8.521577e+12,1.681489e+09,537272027.0,13.509211,4.060675,-2.194613,7.0,722748.1434,3382476.076,130.740482,0.015598,0.026357,-0.996713,0.074998,13.766277,3.001277,-2.299158,0.537272
1,8.603934e+12,8.603941e+12,1.681489e+09,901910094.0,67.571983,8.572300,-1.584964,7.0,722443.6243,3383240.372,120.213521,-0.011228,0.002161,-0.823207,-0.567626,67.668676,5.150399,6.032665,82.901910
2,8.604034e+12,8.604031e+12,1.681489e+09,30675227.0,63.920612,8.279163,-1.499821,7.0,722443.1083,3383241.720,120.263848,-0.011494,0.001544,-0.823327,-0.567450,64.006705,5.020154,5.880340,83.030675
3,8.604134e+12,8.604133e+12,1.681489e+09,117585984.0,60.145790,7.939707,-1.411700,7.0,722442.4885,3383243.337,120.321945,-0.012461,0.001137,-0.823363,-0.567377,60.226942,4.884737,5.603807,83.117586
4,8.604235e+12,8.604245e+12,1.681489e+09,203770432.0,56.400768,7.555552,-1.324138,7.0,722442.0737,3383244.415,120.360779,-0.012506,0.001103,-0.823429,-0.567280,56.478306,4.627558,5.354717,83.203770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,9.018846e+12,9.018845e+12,1.681489e+09,836400577.0,59.419807,4.238417,1.733680,11.0,721826.0000,3389550.000,98.000000,0.000000,0.000000,1.000000,1.000000,59.419807,-1.733680,4.238417,497.836401
300,9.019347e+12,9.019337e+12,1.681489e+09,336471474.0,56.689102,11.029520,2.352881,11.0,721827.0000,3389551.000,98.000000,0.000000,0.000000,1.000000,1.000000,56.689102,-2.352881,11.029520,498.336471
301,9.019648e+12,9.019637e+12,1.681489e+09,601597892.0,55.130283,16.225712,1.337258,11.0,721827.0000,3389552.000,98.000000,0.000000,0.000000,1.000000,1.000000,55.130283,-1.337258,16.225712,498.601598
302,8.845908e+12,8.845902e+12,1.681489e+09,901733093.0,50.526253,15.408875,-3.385401,7.0,722237.0000,3386906.000,111.000000,0.000000,0.000000,-1.000000,-1.000000,50.526253,3.385401,15.408875,324.901733


### number label to type name

In [21]:
# Create a dictionary to map the labels to types
label_to_type = {0: 'person', 2: 'car', 7: 'truck', 9: 'trafficLight', 11: 'stopSign'}

# Add a new column "type" based on the "label" column
ego_obj['type'] = ego_obj['label'].map(label_to_type)

ego_obj

,obj_timestamp,ego_timestamp,secs,nsecs,obj_x,obj_y,obj_z,label,ego_x,ego_y,ego_z,ego_ox,ego_oy,ego_oz,ego_ow,obj_x_t,obj_y_t,obj_z_t,time,type
0,8.521573e+12,8.521577e+12,1.681489e+09,537272027.0,13.509211,4.060675,-2.194613,7.0,722748.1434,3382476.076,130.740482,0.015598,0.026357,-0.996713,0.074998,13.766277,3.001277,-2.299158,0.537272,truck
1,8.603934e+12,8.603941e+12,1.681489e+09,901910094.0,67.571983,8.572300,-1.584964,7.0,722443.6243,3383240.372,120.213521,-0.011228,0.002161,-0.823207,-0.567626,67.668676,5.150399,6.032665,82.901910,truck
2,8.604034e+12,8.604031e+12,1.681489e+09,30675227.0,63.920612,8.279163,-1.499821,7.0,722443.1083,3383241.720,120.263848,-0.011494,0.001544,-0.823327,-0.567450,64.006705,5.020154,5.880340,83.030675,truck
3,8.604134e+12,8.604133e+12,1.681489e+09,117585984.0,60.145790,7.939707,-1.411700,7.0,722442.4885,3383243.337,120.321945,-0.012461,0.001137,-0.823363,-0.567377,60.226942,4.884737,5.603807,83.117586,truck
4,8.604235e+12,8.604245e+12,1.681489e+09,203770432.0,56.400768,7.555552,-1.324138,7.0,722442.0737,3383244.415,120.360779,-0.012506,0.001103,-0.823429,-0.567280,56.478306,4.627558,5.354717,83.203770,truck
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,9.018846e+12,9.018845e+12,1.681489e+09,836400577.0,59.419807,4.238417,1.733680,11.0,721826.0000,3389550.000,98.000000,0.000000,0.000000,1.000000,1.000000,59.419807,-1.733680,4.238417,497.836401,stopSign
300,9.019347e+12,9.019337e+12,1.681489e+09,336471474.0,56.689102,11.029520,2.352881,11.0,721827.0000,3389551.000,98.000000,0.000000,0.000000,1.000000,1.000000,56.689102,-2.352881,11.029520,498.336471,stopSign
301,9.019648e+12,9.019637e+12,1.681489e+09,601597892.0,55.130283,16.225712,1.337258,11.0,721827.0000,3389552.000,98.000000,0.000000,0.000000,1.000000,1.000000,55.130283,-1.337258,16.225712,498.601598,stopSign
302,8.845908e+12,8.845902e+12,1.681489e+09,901733093.0,50.526253,15.408875,-3.385401,7.0,722237.0000,3386906.000,111.000000,0.000000,0.000000,-1.000000,-1.000000,50.526253,3.385401,15.408875,324.901733,truck


In [26]:
ego_obj_with_time = pd.DataFrame({'time': ego_obj['time'],
                                  'obj_utm_x': ego_obj['ego_x'] + ego_obj['obj_y_t'],
                                  'obj_utm_y': ego_obj['ego_y'] + ego_obj['obj_x_t'],
                                  'obj_utm_z': ego_obj['ego_z'] + ego_obj['obj_z_t'],
                                  'ego_utm_x': ego_obj['ego_x'],
                                  'ego_utm_y': ego_obj['ego_y'],
                                  'ego_utm_z': ego_obj['ego_z'],
                                  'type': ego_obj['type'],    
                                  'confidence':0.9                              
                                  })

# Sort the DataFrame by the time column
ego_obj_with_time_sorted = ego_obj_with_time.sort_values('time')

ego_obj_with_time_sorted

,time,obj_utm_x,obj_utm_y,obj_utm_z,ego_utm_x,ego_utm_y,ego_utm_z,type,confidence
0,0.537272,722751.144677,3.382490e+06,128.441324,722748.1434,3382476.076,130.740482,truck,0.9
196,0.537272,722751.101520,3.382490e+06,128.445623,722748.1434,3382476.076,130.740482,truck,0.9
1,82.901910,722448.774699,3.383308e+06,126.246186,722443.6243,3383240.372,120.213521,truck,0.9
197,82.901910,722448.694658,3.383308e+06,126.035453,722443.6243,3383240.372,120.213521,truck,0.9
2,83.030675,722448.128454,3.383306e+06,126.144188,722443.1083,3383241.720,120.263848,truck,0.9
...,...,...,...,...,...,...,...,...,...
193,498.516583,721826.883552,3.389571e+06,103.134929,721827.0000,3389552.000,98.000000,stopSign,0.9
301,498.601598,721825.662742,3.389607e+06,114.225712,721827.0000,3389552.000,98.000000,stopSign,0.9
286,498.601598,721825.676141,3.389607e+06,114.263098,721827.0000,3389552.000,98.000000,stopSign,0.9
194,498.601598,721827.000000,3.389571e+06,103.736700,721827.0000,3389552.000,98.000000,stopSign,0.9


In [27]:
ego_obj_with_time_sorted.to_csv('obj_ego_with_time.csv', index=False)

## Prepare for tracking code

In [52]:
import pandas as pd
import os

# Define the input file path and folder for saving the frames
input_file = "obj_ego_with_time.csv"
output_folder = "yolo_frames/"

# Specify the columns to save in each text file
columns_to_save = ['type', 'confidence', 'obj_utm_x', 'obj_utm_y', 'obj_utm_x', 'obj_utm_y']

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Define the rounding interval
rounding_interval = 0.1

# Round the time column to the nearest multiple of the rounding interval
df['rounded_time'] = (df['time'] // rounding_interval) * rounding_interval

# Generate a continuous range of rounded times
min_time = df['rounded_time'].min()
max_time = df['rounded_time'].max()
rounded_times = pd.Series([i * rounding_interval for i in range(int(min_time * (1 / rounding_interval)), int(max_time * (1 / rounding_interval)) + 1)])

# Iterate over the rounded times and save each frame to a separate text file
for time in rounded_times:
    # Generate the output file path
    output_file = os.path.join(output_folder, f"{time:.1f}.txt")

    # Filter the DataFrame for the current rounded time
    frame_data = df[df['rounded_time'] == time]

    if frame_data.empty:
            # Save an empty file with 'N/A'
            with open(output_file, 'w') as file:
                # file.write('N/A ' * len(columns_to_save))
                # file.write('N/A N/A N/A N/A N/A N/A' )
                file.write('')
    else:
            # Save the frame data to the output file
            frame_data[columns_to_save].to_csv(output_file, sep=' ', index=False, header=False)
